In [1]:
type ∨[P, Q] = Either[P, Q]
type ∧[P, Q] = Tuple2[P, Q]
type ⟶[P, Q] = P => Q
type ⊥ = Nothing
type True = Unit
type ¬[P] = P => ⊥

defined type ∨
defined type ∧
defined type ⟶
defined type ⊥
defined type True
defined type ¬

In [2]:
type Or[P, Q] = Either[P, Q]
type And[P, Q] = Tuple2[P, Q]
type Implies[P, Q] = P => Q
type False = Nothing
type True = Unit
type Not[P] = P => ⊥

defined type Or
defined type And
defined type Implies
defined type False
defined type True
defined type Not

In [3]:
class Inhabitant{ x => 
    // Knight(x) -- `x` is a Knight
    // 
    type Knight
    
    // Knave(x) -- `x` is a Knave (i.e. is not a Knight)
    // 
    type Knave = ¬[Knight]
    
    // Says(x, P) -- `x` says that `P` holds, i.e. asserts proposition `P`
    // 
    type Says[P]
}

defined class Inhabitant

In [4]:
trait KnightsKnaves{
    // P1. Inhabitants are knights or knaves
    // 
    //     ∀ x. Inhabitant(x) ⟶ Knight(x) ∨ Knave(x) 
    // 
    def P1(x: Inhabitant): x.Knight ∨ x.Knave
    
    // In Scala 3
    // val P1: (x: Inhabitant) => Either[x.Knight, x.Knave]
    
    // P2. Knights are truth tellers
    // 
    //     ∀ P. ∀ x. Knight(x) ⟶ Says(x, P) ⟶ P
    // 
    def P2[P](x: Inhabitant): x.Knight => x.Says[P] => P
    
    // In Scala 3
    // val P2: [P] => (x: Inhabitant) => x.Knight => x.Says[P] => P
    
    // P3. Knaves are persistent liers
    // 
    //     ∀ P. ∀ x. Knight(x) ⟶ Says(x, P) ⟶ ¬P
    // 
    def P3[P](x: Inhabitant): x.Knave => x.Says[P] => ¬[P]
}

defined trait KnightsKnaves

# Puzzle 1 - Ejemplo.
Is it possible for any inhabitant of this island to claim that he is a knave?


No; no inhabitant can claim to be a knave because no knight would lie and say he is a knave and no knave would truthfully admit to being a knave.

In [5]:
// { P1, P2, P3 } ⊢ ∀ x. Inhabitant(x) ⟶ ¬Says(x, Knave(x))

def puzzle1(premises: KnightsKnaves)(x: Inhabitant): ¬[x.Says[x.Knave]] = 
    // 1. Says(x, Knave(x))                                       ; hypothesis
    ((_1: x.Says[x.Knave]) => 
         (premises.P1(x) match {
             // 2. Knight(x)                                      ; hypothesis
             case Left(_2: x.Knight) => 
                 //   3. Knight(x) → Says(x, Knave(x)) → Knave(x) ; P2[Knave(x),x]
                 val _3: x.Knight => x.Says[x.Knave] => x.Knave = premises.P2[x.Knave](x)
                 //   4. Says(x, Knave(x)) → Knave(x)             ; ⟶E(3,2)
                 val _4: x.Says[x.Knave] => x.Knave = _3(_2)
                 //   5. Knave(x)                                 ; ⟶E(4,1)
                 //   5. ¬ Knight(x)                              ; ≝ Knave
                 //   5. Knight(x) → ⊥                            ; ≝ ¬ 
                 val _5: x.Knight => ⊥ = _4(_1)
                 //   6. ⊥                                        ; ⟶E(5,2)
                 _5(_2) : ⊥

             //  _7. Knave(x)                                     ; hypothesis
             case Right(_7: x.Knave) => 
                 //   8. Knave(x) → Says(x, Knave(x)) → ¬ Knave(x) ; P3[Knave(x),x]
                 val _8: x.Knave => x.Says[x.Knave] => ¬[x.Knave] = premises.P3[x.Knave](x)
                 //   9. Says(x, Knave(x)) → ¬ Knave(x)            ; ⟶E(8,7)
                 val _9: x.Says[x.Knave] => ¬[x.Knave] = _8(_7)
                 //   10. ¬ Knave(x)                               ; ⟶E(9,1)
                 //   10. Knave(x) → ⊥                             ; ≝ ¬ 
                 val _10: x.Knave => ⊥ = _9(_1)
                 //  11. ⊥                                         ; ⟶E(10,7)
                 _10(_7) : ⊥

        // _12. ⊥                                                  ; ∨E(P1, 2-6, 7-11)
        }) : False

    // _13. Says(x, Knave(x)) ⟶ ⊥                                  ; ⟶I(1-12)
    // _13. ¬ Says(x, Knave(x))                                    ; ≝ ¬ 
    ) : ¬[x.Says[x.Knave]]

defined function puzzle1

In [6]:
// { P1, P2, P3 } ⊢ ∀ x. Inhabitant(x) ⟶ ¬Says(x, Knave(x))

def puzzle1(premises: KnightsKnaves)(x: Inhabitant): x.Says[x.Knave] => Nothing =
    // 1. Says(x, Knave(x))                                       ; hypothesis
    ((_1: x.Says[x.Knave]) => 
         (premises.P1(x) match {
             // 2. Knight(x)                                      ; hypothesis
             case Left(_2: x.Knight) => 
                 //   3. Knight(x) → Says(x, Knave(x)) → Knave(x) ; P2[Knave(x),x]
                 val _3: x.Knight => x.Says[x.Knave] => x.Knave = premises.P2[x.Knave](x)
                 //   4. Says(x, Knave(x)) → Knave(x)             ; ⟶E(3,2)
                 val _4: x.Says[x.Knave] => x.Knave = _3(_2)
                 //   5. Knave(x)                                 ; ⟶E(4,1)
                 //   5. ¬ Knight(x)                              ; ≝ Knave
                 //   5. Knight(x) → ⊥                            ; ≝ ¬ 
                 val _5: x.Knight => ⊥ = _4(_1)
                 //   6. ⊥                                        ; ⟶E(5,2)
                 _5(_2) : ⊥

             //  _7. Knave(x)                                     ; hypothesis
             case Right(_7: x.Knave) => 
                 //   8. Knave(x) → Says(x, Knave(x)) → ¬ Knave(x) ; P3[Knave(x),x]
                 val _8: x.Knave => x.Says[x.Knave] => ¬[x.Knave] = premises.P3[x.Knave](x)
                 //   9. Says(x, Knave(x)) → ¬ Knave(x)            ; ⟶E(8,7)
                 val _9: x.Says[x.Knave] => ¬[x.Knave] = _8(_7)
                 //   10. ¬ Knave(x)                               ; ⟶E(9,1)
                 //   10. Knave(x) → ⊥                             ; ≝ ¬ 
                 val _10: x.Knave => ⊥ = _9(_1)
                 //  11. ⊥                                         ; ⟶E(10,7)
                 _10(_7) : ⊥

        // _12. ⊥                                                  ; ∨E(P1, 2-6, 7-11)
        }) : False

    // _13. Says(x, Knave(x)) ⟶ ⊥                                  ; ⟶I(1-12)
    // _13. ¬ Says(x, Knave(x))                                    ; ≝ ¬ 
    ) : ¬[x.Says[x.Knave]]

defined function puzzle1

In [7]:
implicit class SomeSugar(P: KnightsKnaves){    
    def eitherKnightOrKnave(x: Inhabitant): x.Knight Or x.Knave = P.P1(x)
    def knightsAreTruthTellers[P](x: Inhabitant) = P.P2[P](x)
    def knavesAreLiers[P](x: Inhabitant) = P.P3[P](x)
    
    def noKnightLies[P](x: Inhabitant): x.Knight => x.Says[Not[P]] => P => False = 
        xIsKnight => xSaysNotP => p => 
            P.P2[Not[P]](x)(xIsKnight)(xSaysNotP)(p)
    
    def noKnaveTellsTruth[P](x: Inhabitant): x.Knave => x.Says[P] => P => False = 
        xIsKnave => xSaysP => p => 
            P.P3[P](x)(xIsKnave)(xSaysP)(p)
}

defined class SomeSugar

In [8]:
// { P1, P2, P3 } ⊢ ∀ x. Inhabitant(x) ⟶ ¬ Says(x, Knave(x))

def puzzle1(premises: KnightsKnaves)(x: Inhabitant): Not[x.Says[x.Knave]] =
    xSaysIsKnave =>
        premises.eitherKnightOrKnave(x).fold(
            xIsKnight => 
                // no knight would lie
                premises.noKnightLies(x)(xIsKnight)(xSaysIsKnave)(xIsKnight),
            xIsKnave => 
                // no knave would tell the truth
                premises.noKnaveTellsTruth(x)(xIsKnave)(xSaysIsKnave)(xIsKnave)
        )

defined function puzzle1

# Puzzles Introducción

Primero se consideran 5 preguntas que servirán como introduccion a la logica 
knight-knave para aquellos que no estén familiarizados con ella y como un
breve curso recordatorio para los que lo están.

#### 2. Is it possible for an inhabitant of the island to claim that he and his brother are both knaves?

This question has provoked a good deal of controversy! Some claim that anyone who says that he and his brother are both knaves is certainly claiming that he is a knave, which is not possible, as we have seen in the answer to Question 1. Therefore, they conclude, no inhabitant can claim that he and his brother are both knaves.

This argument is wrong! Suppose an inhabitant A is a knave and his brother B is a knight. Then it is false that he and his brother are both knaves, hence he, as a knave, is certainly capable of making that false statement. Therefore it is possible for an inhabitant to claim that he and his brother are both knaves, but only if he is a knave and his brother is a knight.

This illustrates a curious principle about the logic of lying and truth-telling: Normally, if a truthful person claims that both of two statements are true, then he will certainly claim that each of the statements is true separately. But with a constant liar, the matter is different. Consider the following two statements: (1) My brother is a knave; (2) I am a knave. A
knave could claim that (1) and (2) together are both true, provided his brother is actually a knight, but he cannot claim (1) and claim (2)  separately, since he cannot claim (2). Again, a knave could say: "I am a knave and two plus two is five, " but he cannot separately claim: (1) "I am a knave"; (2) "Two plus two is five."

In [8]:
/* 
Es posible para un habitante de la isla afirmar que él y su hermano son ambos knaves?¿

Si A es un knave y su hermano B es un Knight:
El predicado A y B son ambos knaves es FALSO, pero al ser A un knave (que siempre miente), podria afirmar que A y B 
son ambos knaves

Por tanto la signatura es:

x.dice[(x.knave and y.knave)] => (se cumple solo si) x.knave and y.knight

x.Says[(x.Knave,y.Knave)] => (x.Knave,y.Knight)

*/

In [15]:
def puzzle2(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): x.Says[And[x.Knave, y.Knave]] = ???

defined function puzzle2

In [16]:
def puzzle2(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): x.Says[x.Knave And y.Knave] => x.Knave And y.Knight = ???

defined function puzzle2

In [15]:
def puzzle2(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): x.Says[(x.Knave,y.Knave)] => (x.Knave,y.Knight) = 
    ({xSay: x.Says[(x.Knave,y.Knave)] => 
        (premises.P1(x) match{
            case Left(xKnight: x.Knight) =>
                // def P2[P](x: Inhabitant): x.Knight => x.Says[P] => P
                val x_L1: x.Knight => x.Says[(x.Knave,y.Knave)] => (x.Knave,y.Knave) = premises.P2[(x.Knave,y.Knave)](x)
                val x_L2: x.Says[(x.Knave,y.Knave)] => (x.Knave,y.Knave) = x_L1(xKnight)
                val x_L3: (x.Knave,y.Knave) = x_L2(xSay)
                x_L3._1: x.Knave
                x_L3._2: y.Knave
                
                val x_L4: x.Knight => Nothing = x_L3._1
                (x_L3._1, x_L4(xKnight)) : (x.Knave,y.Knight)
            
            case Right(xKnave: x.Knave) =>
                
                //def P3[P](x: Inhabitant): x.Knave => x.Says[P] => ¬[P]
                val x_R1: x.Knave => x.Says[(x.Knave,y.Knave)] => ¬[(x.Knave,y.Knave)] = premises.P3[(x.Knave,y.Knave)](x)
                val x_R2: x.Says[(x.Knave,y.Knave)] => ¬[(x.Knave,y.Knave)] = x_R1(xKnave)
                //val x_R3: ¬[(x.Knave,y.Knave)] = x_R2(xSay)
                val x_R3: ((x.Knave,y.Knave)) => Nothing = x_R2(xSay)
                
                //Para usar la funcion x_R3 necesito una tupla con x.Knave que ya lo tengo y con y.Knave que no lo tengo
                //Por eso necesito usar la P1 con (y) y poder sacar el y.Knave
            
                (premises.P1(y) match{
                    // Siempre que se hace un pattern matching hay que poner ambas ramas
                    case Left(yKnight: y.Knight) => (xKnave, yKnight) : (x.Knave,y.Knight)
                    // En el caso de Left, ya tengo el y.Knight asi que he obtenido la tupla (x.Knave,y.Knight)
                    case Right(yKnave: y.Knave) => x_R3((xKnave, yKnave)) : (x.Knave,y.Knight)
                    // En el caso del Right, solo tengo el x.Knave pero NO el y.Knight, pero puedo obtener un Nothing 
                    // en lugar de la tupla (x.Knave,y.Knight), porque el tipo Nothing es equivalente al tipo (x.Knave,y.Knight)
                }): (x.Knave,y.Knight) // Esta definicion de datos se puede quitar
                
        }): (x.Knave,y.Knight)
    })

defined function puzzle2

In [20]:
//SIMPLIFICADO
def puzzle2(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): x.Says[(x.Knave,y.Knave)] => (x.Knave,y.Knight) = 
    ({xSay => 
        (premises.P1(x) match{
            case Left(xKnight) =>
                val x_L1: x.Knight => x.Says[(x.Knave,y.Knave)] => (x.Knave,y.Knave) = premises.P2[(x.Knave,y.Knave)](x)
                val x_L2: x.Says[(x.Knave,y.Knave)] => (x.Knave,y.Knave) = x_L1(xKnight)
                val x_L3: (x.Knave,y.Knave) = x_L2(xSay)
                val x_L4: x.Knight => Nothing = x_L3._1 //Esto se puede obviar
                (x_L3._1, x_L4(xKnight)) //Obviando x_L4, podemos sacar x_L3._1(xKnight) : (x.Knave,y.Knight)
                (x_L3._1, x_L4(xKnight)) // x_L3._1(xKnight) : (x.Knave,y.Knight)
                
            
            case Right(xKnave) =>
                val x_R1: x.Knave => x.Says[(x.Knave,y.Knave)] => ¬[(x.Knave,y.Knave)] = premises.P3[(x.Knave,y.Knave)](x)
                val x_R2: x.Says[(x.Knave,y.Knave)] => ¬[(x.Knave,y.Knave)] = x_R1(xKnave)
                val x_R3: ((x.Knave,y.Knave)) => Nothing = x_R2(xSay)
                
                premises.P1(y) match{
                    case Left(yKnight) => (xKnave, yKnight) 
                    case Right(yKnave) => x_R3((xKnave, yKnave)) 
                }
                
        })
    })

defined function puzzle2

In [21]:
//SIMPLIFICADO - NUEVA DEFINICION DE DATOS

def puzzle2(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): x.Says[(x.Knave,y.Knave)] => (x.Knave,y.Knight) = 
    ({xSay => 
        (premises.P1(x) match{
            case Left(xKnight) =>
                //val x_L1: x.Knight => x.Says[(x.Knave,y.Knave)] => (x.Knave,y.Knave) = premises.P2[(x.Knave,y.Knave)](x)
                val x_L1 = premises.P2[(x.Knave,y.Knave)](x) : (x.Knight => x.Says[(x.Knave,y.Knave)] => (x.Knave,y.Knave))
                //val x_L2: x.Says[(x.Knave,y.Knave)] => (x.Knave,y.Knave) = x_L1(xKnight)
                val x_L2 = x_L1(xKnight) : (x.Says[(x.Knave,y.Knave)] => (x.Knave,y.Knave))
                //val x_L3: (x.Knave,y.Knave) = x_L2(xSay)
                val x_L3 = x_L2(xSay) : (x.Knave,y.Knave) // Equivale a val x_L3 = x_L1(xKnight)(xSay) : (x.Knave,y.Knave)
                //val x_L4: x.Knight => Nothing = x_L3._1 //Esto se puede obviar
                //(x_L3._1, x_L4(xKnight)) //Obviando x_L4, podemos sacar x_L3._1(xKnight) : (x.Knave,y.Knight)
                //(x_L3._1, x_L4(xKnight)) // x_L3._1(xKnight) : (x.Knave,y.Knight)
                
                x_L3._1(xKnight) : (x.Knave,y.Knight)
            
            case Right(xKnave) =>
                //val x_R1: x.Knave => x.Says[(x.Knave,y.Knave)] => ¬[(x.Knave,y.Knave)] = premises.P3[(x.Knave,y.Knave)](x)
                val x_R1 = premises.P3[(x.Knave,y.Knave)](x) : (x.Knave => x.Says[(x.Knave,y.Knave)] => ¬[(x.Knave,y.Knave)])
                //val x_R2: (x.Says[(x.Knave,y.Knave)] => ¬[(x.Knave,y.Knave)]) = x_R1(xKnave)
                val x_R2 = x_R1(xKnave) : (x.Says[(x.Knave,y.Knave)] => ¬[(x.Knave,y.Knave)])
                //val x_R3: ((x.Knave,y.Knave)) => Nothing = x_R2(xSay)
                val x_R3 = x_R2(xSay) : (((x.Knave,y.Knave)) => Nothing)
                
                premises.P1(y) match{
                    case Left(yKnight) => (xKnave, yKnight) 
                    case Right(yKnave) => x_R3((xKnave, yKnave)) 
                }
                
        })
    })

defined function puzzle2

In [22]:
// Sugar version

def puzzle2(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): x.Says[(x.Knave,y.Knave)] => (x.Knave,y.Knight) = 
    xSays =>
        premises.eitherKnightOrKnave(x).fold(           
            xIsKnight => 
                (premises.knightsAreTruthTellers[(x.Knave,y.Knave)](x)(xIsKnight)(xSays))._1(xIsKnight)
            ,
            xIsKnave => {
                premises.eitherKnightOrKnave(y).fold(
                    yIsKnight => 
                        (xIsKnave, yIsKnight),
                    yIsKnave =>
                        premises.knavesAreLiers[(x.Knave,y.Knave)](x)(xIsKnave)(xSays)(xIsKnave, yIsKnave)
                )
            }
        )

defined function puzzle2